# Wuauclt CreateRemoteThread Execution

## Metadata


|                   |    |
|:------------------|:---|
| collaborators     | ['@Cyb3rWard0g'] |
| creation date     | 2020/10/12 |
| modification date | 2020/10/12 |
| playbook related  | [] |

## Hypothesis
Adversaries might be proxy executing code via the Windows Update client utility in my environment and creating and running a thread in the virtual address space of another process via the CreateRemoteThread API to bypass rules looking for it calling out to the Internet.

## Technical Context
The Windows Update client (wuauclt.exe) utility allows you some control over the functioning of the Windows Update Agent.

## Offensive Tradecraft
Adversaries can leverage this utility to proxy the execution of code by specifying an arbitrary DLL with the following command line `wuauclt.exe /UpdateDeploymentProvider <Full_Path_To_DLL> /RunHandlerComServer`

## Mordor Test Data


|           |           |
|:----------|:----------|
| metadata  | https://mordordatasets.com/notebooks/small/windows/05_defense_evasion/SDWIN-201012183248.html        |
| link      | [https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/defense_evasion/host/covenant_lolbin_wuauclt_createremotethread.zip](https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/defense_evasion/host/covenant_lolbin_wuauclt_createremotethread.zip)  |

## Analytics

### Initialize Analytics Engine

In [ ]:
from openhunt.mordorutils import *
spark = get_spark()

### Download & Process Mordor Dataset

In [ ]:
mordor_file = "https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/defense_evasion/host/covenant_lolbin_wuauclt_createremotethread.zip"
registerMordorSQLTable(spark, mordor_file, "mordorTable")

### Analytic I
Look for wuauclt with the specific parameters used to load and execute a DLL.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| Process | Microsoft-Windows-Sysmon/Operational | Process created Process | 1 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, Image, CommandLine
FROM mordorTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
    AND EventID = 1
    AND Image LIKE '%wuauclt.exe'
    AND CommandLine LIKE '%wuauclt%UpdateDeploymentProvider%.dll%RunHandlerComServer'
'''
)
df.show(10,False)

### Analytic II
Look for unsigned DLLs being loaded by wuauclt. You might have to stack the results and find potential anomalies over time.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| Module | Microsoft-Windows-Sysmon/Operational | Process loaded DLL | 7 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, Image, ImageLoaded
FROM mordorTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
    AND EventID = 7
    AND Image LIKE '%wuauclt.exe'
    AND Signed = 'false'
'''
)
df.show(10,False)

### Analytic III
Look for wuauclt creating and running a thread in the virtual address space of another process via the CreateRemoteThread API.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| Process | Microsoft-Windows-Sysmon/Operational | Process wrote_to Process | 8 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, TargetImage
FROM mordorTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
    AND EventID = 8
    AND SourceImage LIKE '%wuauclt.exe'
'''
)
df.show(10,False)

### Analytic IV
Look for recent files created being loaded by wuauclt.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Sysmon/Operational | Process created File | 11 |
| File | Microsoft-Windows-Sysmon/Operational | Process loaded DLL | 7 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ImageLoaded
FROM mordorTable b
INNER JOIN (
    SELECT TargetFilename, ProcessGuid
    FROM mordorTable
    WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
        AND EventID = 11
    ) a
ON b.ImageLoaded = a.TargetFilename
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
  AND EventID = 7
  AND Image LIKE '%wuauclt.exe'
'''
)
df.show(10,False)

### Analytic V
Look for wuauclt loading recently created DLLs and writing to another process


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| Module | Microsoft-Windows-Sysmon/Operational | Process created File | 11 |
| Module | Microsoft-Windows-Sysmon/Operational | Process loaded DLL | 7 |
| Module | Microsoft-Windows-Sysmon/Operational | Process wrote_to Process | 8 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, d.TargetImage, c.ImageLoaded
FROM mordorTable d
INNER JOIN (
    SELECT b.ProcessGuid, b.ImageLoaded
    FROM mordorTable b
    INNER JOIN (
      SELECT TargetFilename, ProcessGuid
      FROM mordorTable
      WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
          AND EventID = 11
    ) a
    ON b.ImageLoaded = a.TargetFilename
    WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
      AND EventID = 7
      AND Image LIKE '%wuauclt.exe'
) c
ON d.SourceProcessGuid = c.ProcessGuid
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
      AND EventID = 8
      AND SourceImage LIKE '%wuauclt.exe'
'''
)
df.show(10,False)

## Known Bypasses


| Idea | Playbook |
|:-----|:---------|

## False Positives
None

## Hunter Notes
* Baseline your environment to identify normal activity from wuauclt. You will have to do some stacking on images loaded by wuauclt (Split it on signed and un-signed (if applicable))

## Hunt Output

| Type | Link |
| :----| :----|
| Sigma Rule | [https://github.com/Neo23x0/sigma/blob/2cb540f95ed2d02f7d8a2bf84d9bf4a5962f156d/rules/windows/process_creation/sysmon_proxy_execution_wuauclt.yml](https://github.com/Neo23x0/sigma/blob/2cb540f95ed2d02f7d8a2bf84d9bf4a5962f156d/rules/windows/process_creation/sysmon_proxy_execution_wuauclt.yml) |
| Sigma Rule | [https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/sysmon_proxy_execution_wuauclt_createremotethread.yml](https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/sysmon_proxy_execution_wuauclt_createremotethread.yml) |

## References
* https://dtm.uk/wuauclt/
* https://docs.microsoft.com/zh-cn/security-updates/windowsupdateservices/21740782
* https://github.com/Neo23x0/sigma/blob/2cb540f95ed2d02f7d8a2bf84d9bf4a5962f156d/rules/windows/process_creation/sysmon_proxy_execution_wuauclt.yml